In [32]:
from itertools import combinations
import numpy as np
import sys
from oxDNA_analysis_tools.UTILS.oxview import oxdna_conf, from_path
from oxDNA_analysis_tools.UTILS.RyeReader import describe, get_confs, inbox
from oxDNA_analysis_tools.UTILS.data_structures import TopInfo, TrajInfo
from pathlib import Path
import os
from ipy_oxdna.dna_structure import DNAStructure, DNAStructureStrand, load_dna_structure, DNABase, strand_from_info
from copy import deepcopy
from ipy_oxdna.oxdna_simulation import Simulation, SimulationManager
import copy
from tqdm.auto import tqdm
from ipy_oxdna.oxdna_simulation import Simulation, SimulationManager
import os
import random

In [2]:
input_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/unmodified/1512_bp'

def load_dna_structure_files(input_path):
    dat_path = os.path.join(input_path, '1512_bp.dat')
    top_path = os.path.join(input_path, '1512_bp.top')
    dna = load_dna_structure(top_path, dat_path)
    return dna

In [3]:
dna = load_dna_structure_files(input_path)

In [4]:
def find_longest_strand(dna):
    longest_strand = None
    longest_strand_index = -1
    max_length = 0
    for index, strand in enumerate(dna.strands):
        if len(strand.bases) > max_length:
            max_length = len(strand.bases)
            longest_strand = strand
            longest_strand_index = index
    return longest_strand, longest_strand_index

In [5]:
longest_strand, longest_strand_index = list(find_longest_strand(dna))      # Has been checked with the oxview information and they match
print ("longest_strand:",longest_strand)
print("longest_strand_index:",longest_strand_index)
print("Number of bases in longest strand:",len(longest_strand))

longest_strand: <ipy_oxdna.dna_structure.DNAStructureStrand object at 0x717664659010>
longest_strand_index: 2
Number of bases in longest strand: 1512


In [6]:
def find_cross_over_in_longest_strand(longest_strand):
    min_distance = float('inf')
    max_index_difference = 0
    cross_over_bases_max = (None, None)
    num_bases = len(longest_strand)
    for i in range(num_bases):
        for j in range(i + 1, num_bases):
            base_i = longest_strand[i]
            base_j = longest_strand[j]
            index_difference = abs(base_i.uid - base_j.uid)
            distance = np.linalg.norm(np.array(base_i.pos) - np.array(base_j.pos))
            if index_difference > max_index_difference or (index_difference == max_index_difference and distance < min_distance):
                max_index_difference = index_difference
                min_distance = distance
                cross_over_bases_max = (base_i, base_j)
    return cross_over_bases_max, max_index_difference, min_distance

In [7]:
cross_over_bases_max, max_index_difference, min_distance = find_cross_over_in_longest_strand(longest_strand)          # Has been checked with the oxview info and they match
print("cross_over_bases_max:",cross_over_bases_max)
print("max_index_difference:", max_index_difference)
print("min_distance:", min_distance)

cross_over_bases_max: (DNABase(uid=43, base='T', pos=array([ 2.10507846, -0.6867429 ,  0.12683482]), a1=array([ 0.17443815, -0.98466813,  0.        ]), a3=array([0., 0., 1.])), DNABase(uid=1554, base='T', pos=array([2.37799716, 0.69653118, 0.12683482]), a1=array([-0.28042587,  0.95987568,  0.        ]), a3=array([ 0.,  0., -1.])))
max_index_difference: 1511
min_distance: 1.4099403509855792


In [8]:
def calculate_left_right_pos(dna, left_indices, right_indices):
    left_pos = []
    right_pos = []
    
    for strand in dna.strands:
        for base in strand:
            if base.uid in left_indices:
                left_pos.append(base.pos)
            elif base.uid in right_indices:
                right_pos.append(base.pos)
    
    if left_pos:
        cms_left_side = np.mean(left_pos, axis=0)
    else:
        raise ValueError("No positions found for left indices.")
    
    if right_pos:
        cms_right_side = np.mean(right_pos, axis=0)
    else:
        raise ValueError("No positions found for right indices.")
    
    print(f"Center of mass for the left side: {cms_left_side}")
    print(f"Center of mass for the right side: {cms_right_side}")
    
    midpoint = (cms_left_side + cms_right_side) / 2
    print(f"Midpoint between the left and right sides: {midpoint}")
    
    return cms_left_side, cms_right_side, midpoint

In [9]:
def is_point_far_from_crossovers(point, crossover_positions, min_distance_threshold):
    for pos in crossover_positions:
        distance = np.linalg.norm(np.array(point) - np.array(pos))
        if distance < min_distance_threshold:
            return False
    return True

In [10]:

def find_valid_point(dna, left_indices, right_indices, longest_strand, min_distance_threshold=4):
    cms_left_side, cms_right_side, midpoint = calculate_left_right_pos(dna, left_indices, right_indices)
    
    cross_over_bases, max_index_difference, min_distance = find_cross_over_in_longest_strand(longest_strand)
    crossover_positions = [base.pos for base in cross_over_bases if base is not None]
    
    t = random.uniform(0, 1)
    first_P = np.array(cms_left_side + t * (cms_right_side - cms_left_side))
    if not crossover_positions:
        return first_P
    
    while True:
        t = random.uniform(0, 1)
        P = np.array(cms_left_side + t * (cms_right_side - cms_left_side))
        if is_point_far_from_crossovers(P, crossover_positions, min_distance_threshold):
            return P

In [11]:
# Find some point on both left and right sides of the point P to calculate the bent angle 
def find_bases_around_point(dna, point, min_distance, max_distance):
    left_bases = []
    right_bases = []
    left_base_indices = []
    left_strand_indices = []
    right_base_indices = []
    right_strand_indices = []
    
    for strand_index, strand in enumerate(dna.strands):
        for base_index, base in enumerate(strand):
            distance = np.linalg.norm(np.array(base.pos) - np.array(point))
            if min_distance < distance < max_distance:
                if base.pos[0] < point[0]:
                    left_bases.append(base.pos)
                    left_base_indices.append(base_index)
                    left_strand_indices.append(strand_index)
                else:
                    right_bases.append(base.pos)
                    right_base_indices.append(base_index)
                    right_strand_indices.append(strand_index)
                    
   
    if left_bases:
        cms_left_bases = np.mean(left_bases, axis=0)
        print(f"Center of mass for left bases around: {cms_left_bases}")
    else:
        cms_left_bases = None
        print("No left bases found.")
    
    if right_bases:
        cms_right_bases = np.mean(right_bases, axis=0)
        print(f"Center of mass for right bases around: {cms_right_bases}")
    else:
        cms_right_bases = None
        print("No right bases found.")
    
    return (left_bases, right_bases, 
            cms_left_bases, cms_right_bases, 
            left_base_indices, right_base_indices, 
            left_strand_indices, right_strand_indices)


In [12]:

P = find_valid_point(dna, left_indices, right_indices, longest_strand)
print(f"Valid point P: {P}")


Center of mass for the left side: [ 3.04825814e-03 -1.26263565e-01 -3.68938322e+01]
Center of mass for the right side: [2.03818308e-02 1.48186293e-01 3.82790798e+01]
Midpoint between the left and right sides: [0.01171504 0.01096136 0.69262384]
Valid point P: [ 7.07853484e-03 -6.24504730e-02 -1.94151714e+01]


In [13]:
(left_bases, right_bases, 
 cms_left_bases, cms_right_bases, 
 left_base_indices, right_base_indices, 
 left_strand_indices, right_strand_indices) = find_bases_around_point(dna, P, min_distance, max_distance)
print(f"Left bases: {left_bases}")
print(f"Right bases: {right_bases}")
print(f"Center of mass for left bases: {cms_left_bases}")
print(f"Center of mass for right bases: {cms_right_bases}")
print(f"Indices of left bases: {left_base_indices}")
print(f"Indices of right bases: {right_base_indices}")
print(f"Indices of strands for left bases: {left_strand_indices}")
print(f"Indices of strands for right bases: {right_strand_indices}")

Center of mass for left bases around: [-1.58697625e+00 -2.95165819e-03 -1.94252991e+01]
Center of mass for right bases around: [ 1.58793792e+00  4.93490100e-03 -1.94568001e+01]
Left bases: [array([ -2.72855115,   0.97585857, -13.125103  ]), array([ -2.80469251,   1.32487285, -13.51486588]), array([ -2.66896558,   1.65530956, -13.90462875]), array([ -2.36948133,   1.85003459, -14.29439163]), array([ -2.07347226,   1.85741222, -14.68415451]), array([ -1.75976682,   1.67244899, -15.07391739]), array([ -1.61081433,   1.34012783, -15.46368027]), array([ -1.68149304,   0.98287761, -15.85344315]), array([ -1.94576001,   0.73230922, -16.24320602]), array([ -2.30626416,   0.68072689, -16.6329689 ]), array([ -2.630193  ,   0.84713924, -17.02273178]), array([ -2.783746  ,   1.10708034, -17.41249466]), array([ -2.77552152,   1.46420848, -17.80225754]), array([ -2.56577039,   1.75336635, -18.19202042]), array([ -2.22884107,   1.87206066, -18.58178329]), array([ -1.88416791,   1.77821147, -18.971546

In [14]:
def calculate_center_of_mass(positions):
    if not positions:
        raise ValueError("No positions provided for center of mass calculation.")
    return np.mean(positions, axis=0)

In [15]:
def calculate_bend_angle(P, cms_left, cms_right):
    vec_left = cms_left - P
    vec_right = cms_right - P
    unit_vec_left = vec_left / np.linalg.norm(vec_left)
    unit_vec_right = vec_right / np.linalg.norm(vec_right)
    dot_product = np.dot(unit_vec_left, unit_vec_right)
    angle = np.arccos(dot_product) * (180.0 / np.pi)
    return angle

In [16]:
def find_bend_angle(dna, left_indices, right_indices, longest_strand, min_distance_threshold=5.0, min_distance=7.0, max_distance=20.0):
    point_pos = find_valid_point(dna, left_indices, right_indices, longest_strand, min_distance_threshold)
    (left_bases, right_bases, 
    cms_left_bases, cms_right_bases, 
    left_base_indices, right_base_indices, 
    left_strand_indices, right_strand_indices) = find_bases_around_point(dna, P, min_distance, max_distance)  
    cms_left = calculate_center_of_mass(left_bases)
    cms_right = calculate_center_of_mass(right_bases)
    bend_angle = calculate_bend_angle(point_pos, cms_left, cms_right)
    return point_pos, bend_angle

In [17]:
P_pos, bend_angle = find_bend_angle(dna, left_indices, right_indices, longest_strand, min_distance_threshold=5.0, min_distance=7.0, max_distance=20.0)
print(f"Point P position: {P_pos}")
print(f"Bend angle at point P: {bend_angle} degrees")

Center of mass for the left side: [ 3.04825814e-03 -1.26263565e-01 -3.68938322e+01]
Center of mass for the right side: [2.03818308e-02 1.48186293e-01 3.82790798e+01]
Midpoint between the left and right sides: [0.01171504 0.01096136 0.69262384]
Center of mass for left bases around: [-1.60429716e+00  5.79545422e-03 -1.94762260e+01]
Center of mass for right bases around: [ 1.60814113e+00 -9.00671879e-03 -1.94597042e+01]
Point P position: [ 6.75845609e-03 -6.75184165e-02 -2.08033014e+01]
Bend angle at point P: 100.45529758482698 degrees


In [18]:
def calculate_angles_for_all_structures(dna_list, left_indices, right_indices, min_distance_threshold=2.0, min_distance=5.0, max_distance=10.0):
    angles = []
    for dna in dna_list:
        longest_strand, _ = find_longest_strand(dna)
        point_pos, bend_angle = find_bend_angle(dna, left_indices, right_indices, longest_strand, min_distance_threshold, min_distance, max_distance)
        angles.append((point_pos, bend_angle))
    return angles

In [19]:
sphere_radius = 3.5   # I checked the radius of the bundle from the oxview and chose this number for the sphere radius

def find_bases_in_sphere(dna, point, sphere_radius):
    bases_in_sphere = []
    base_to_strand_mapping = {}
    for strand_index, strand in enumerate(dna.strands):
        for base in strand:
            base_position = np.array(base.pos)
            distance = np.linalg.norm(base_position - point)
            if distance < sphere_radius:
                bases_in_sphere.append(base.uid)
                base_to_strand_mapping[base.uid] = strand_index
    return bases_in_sphere, base_to_strand_mapping

In [20]:
bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, P, sphere_radius)
print('bases_in_sphere:', bases_in_sphere)
print('base_to_strand_mapping:', base_to_strand_mapping)
print('Number_bases_in_sphere:', len(bases_in_sphere))

bases_in_sphere: [587, 588, 589, 590, 591, 592, 593, 594, 595, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1232, 1233, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1830, 1831, 1832, 1839, 1840, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565, 2566, 2567, 2568, 2569, 2570, 2571, 2860, 2861, 2862, 2863, 2864, 2865, 2880]
base_to_strand_mapping: {587: 2, 588: 2, 589: 2, 590: 2, 591: 2, 592: 2, 593: 2, 594: 2, 595: 2, 737: 2, 738: 2, 739: 2, 740: 2, 741: 2, 742: 2, 743: 2, 744: 2, 745: 2, 

In [21]:
def export_dna_structures(new_dna_structures, base_path):
    output_paths = []
    for i, new_dna in enumerate(new_dna_structures):
        structure_id = i
        unique_subdir = os.path.join(base_path, f'structure_{structure_id}')
        os.makedirs(unique_subdir, exist_ok=True)
        dat_path = os.path.join(unique_subdir, '1512_bp_rmv_staples.dat')
        top_path = os.path.join(unique_subdir, '1512_bp_rmv_staples.top')
        new_dna.export_top_conf(Path(top_path), Path(dat_path))
        output_paths.append({
            'structure_id': structure_id,
            'dat_path': dat_path,
            'top_path': top_path
        })
    return output_paths

In [22]:
def remove_one_strand_in_sphere(dna, point, sphere_radius):
    
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, point, sphere_radius)
    longest_strand, longest_strand_index = find_longest_strand(dna)
    strands_to_remove = set(base_to_strand_mapping.values()) - {longest_strand_index}
    strands_in_sphere = list(set(base_to_strand_mapping.values()))
    print("Strand indices in the sphere:", strands_in_sphere)
    print("Strand indices to be removed:", list(strands_to_remove))
    dna_structures = []
    for strand_index in strands_to_remove:
        strand_list = []
        for idx, strand in enumerate(dna.strands):
            if idx != strand_index:
                strand_list.append(strand)
        new_dna_structure = DNAStructure(strand_list, dna.time, dna.box, dna.energy)
        dna_structures.append(new_dna_structure)
    return dna_structures

In [23]:
new_dna_structures = remove_one_strand_in_sphere(dna, P, sphere_radius)
print(len(new_dna_structures))

Strand indices in the sphere: [2, 8, 9, 10, 11, 40, 30]
Strand indices to be removed: [8, 9, 10, 11, 40, 30]
6


In [ ]:
angles = calculate_angles_for_all_structures(new_dna_structures, left_indices, right_indices)

for i, (point_pos, bend_angle) in enumerate(angles):
    print(f"Structure {i}: Point P position: {point_pos}, Bend angle: {bend_angle} degrees")

In [24]:
base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd'
output_paths_one = export_dna_structures(new_dna_structures, base_path)

In [25]:
def main_one_strand(base_path, sim_base_path, left_indices, right_indices, rel_parameters, eq_parameters, prod_parameters, target_angle=90, tolerance=5):
    # Load the initial DNA structure
    print("Loading initial DNA structure...")
    initial_dna = load_dna_structure_files(base_path)
    
    results = []
    sphere_radius = 3.5  

    # Find the longest strand in the initial DNA structure
    print("Finding the longest strand in the DNA structure...")
    longest_strand, _ = find_longest_strand(initial_dna)

    # Find a valid point as the center of the sphere
    print("Finding a valid point P as the center of the sphere...")
    point = find_valid_point(initial_dna, left_indices, right_indices, longest_strand)
    print(f"Point P found at: {point}")

    # Find all bases (staples) within the sphere around the point
    print(f"Finding all bases within a sphere of radius {sphere_radius} around point P...")
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(initial_dna, point, sphere_radius)

    # Create a list of unique strands (staples) within the sphere
    strands_in_sphere = set(base_to_strand_mapping.values())
    print(f"Number of unique strands (staples) found in the sphere: {len(strands_in_sphere)}")

    for strand_index in strands_in_sphere:
        print(f"Processing strand index: {strand_index}")
        
        # Create a new structure by removing the current strand
        strand_list = [strand for idx, strand in enumerate(initial_dna.strands) if idx != strand_index]
        new_dna_structure = DNAStructure(strand_list, initial_dna.time, initial_dna.box, initial_dna.energy)

        # Export the new structure and run simulations
        print("Exporting the new structure...")
        output_paths = export_dna_structures([new_dna_structure], base_path)
        
        print("Running all simulations (relaxation, equilibration, production)...")
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_dna_structure], base_path, sim_base_path, rel_parameters, eq_parameters, prod_parameters)
        
        # Find the bend angle of the new structure
        print("Finding the bend angle of the new structure...")
        point_pos, bend_angle = find_bend_angle(new_dna_structure, left_indices, right_indices, longest_strand)
        print(f"Bend angle: {bend_angle} at point {point_pos}")
        
        results.append((point_pos, bend_angle))
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
    
    print("Finished processing all strands.")
    return results

In [ ]:
def remove_two_strands_in_sphere(dna, point, sphere_radius):
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, P, sphere_radius)
    longest_strand, longest_strand_index = find_longest_strand(dna)
    
    print("Bases in sphere:", bases_in_sphere)
    print("Base to strand mapping:", base_to_strand_mapping)
    print("Longest strand:", longest_strand)
    print("Longest strand index:", longest_strand_index)
    
    strands_to_remove = set(base_to_strand_mapping.values()) - {longest_strand_index}
    print("Strands to remove:", strands_to_remove)

    dna_structures = []
    removed_strands_info = []

    # Create all possible pairs of strands to remove
    strand_pairs = [(strand_1, strand_2) for i, strand_1 in enumerate(strands_to_remove)
                    for strand_2 in list(strands_to_remove)[i + 1:]]

    for strand_1, strand_2 in strand_pairs:
        strand_list = []
        for idx, strand in enumerate(dna.strands):
            if idx not in {strand_1, strand_2}:
                strand_list.append(strand)
        
        new_dna_structure = DNAStructure(strand_list, dna.time, dna.box, dna.energy)
        dna_structures.append(new_dna_structure)
        
        removed_strands_info.append((strand_1, strand_2))
        print(f"Removed strands: {strand_1}, {strand_2}")
    
    return dna_structures, removed_strands_info



In [ ]:
new_dna_structures_two, removed_strands_info_two = remove_two_strands_in_sphere(dna, P, sphere_radius)

print(f"Generated {len(new_dna_structures_two)} new DNA structures with two strands removed.")
# print("Details of removed strands for each structure:")
# for strands in removed_strands_info_two:
#     print(f"Strands removed: {strands}")

In [ ]:
new_dna_structures_two, removed_strands_info_two = remove_two_strands_in_sphere(dna, P, sphere_radius)
print(new_dna_structures_two)
print(len(new_dna_structures_two))

In [ ]:
base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/two_staples_remvd'
output_paths_two = export_dna_structures(new_dna_structures_two, base_path)

In [ ]:
def remove_three_strands_in_sphere(dna, point, sphere_radius):
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, P, sphere_radius)
    longest_strand, longest_strand_index = find_longest_strand(dna)
    
    print("Bases in sphere:", bases_in_sphere)
    print("Base to strand mapping:", base_to_strand_mapping)
    print("Longest strand:", longest_strand)
    print("Longest strand index:", longest_strand_index)
    
    strands_to_remove = set(base_to_strand_mapping.values()) - {longest_strand_index}
    print("Strands to remove:", strands_to_remove)

    dna_structures = []
    removed_strands_info = []

    # Create all possible triplets of strands to remove
    strand_triplets = [(strand_1, strand_2, strand_3) for i, strand_1 in enumerate(strands_to_remove)
                       for j, strand_2 in enumerate(list(strands_to_remove)[i + 1:])
                       for strand_3 in list(strands_to_remove)[i + j + 2:]]

    for strand_1, strand_2, strand_3 in strand_triplets:
        strand_list = []
        for idx, strand in enumerate(dna.strands):
            if idx not in {strand_1, strand_2, strand_3}:
                strand_list.append(strand)
        
        new_dna_structure = DNAStructure(strand_list, dna.time, dna.box, dna.energy)
        dna_structures.append(new_dna_structure)
        
        removed_strands_info.append((strand_1, strand_2, strand_3))
        print(f"Removed strands: {strand_1}, {strand_2}, {strand_3}")
    
    return dna_structures, removed_strands_info


In [ ]:
new_dna_structures_three, removed_strands_info = remove_three_strands_in_sphere(dna, P, sphere_radius)
print(f"Generated {len(new_dna_structures_three)} new DNA structures with three strands removed.")
# print("Details of removed strands for each structure:")
# for strands in removed_strands_info:
#     print(f"Strands removed: {strands}")

In [ ]:
new_dna_structures_three, removed_strands_info = remove_three_strands_in_sphere(dna, P, sphere_radius)
print(new_dna_structures_three)
print(len(new_dna_structures_three))

In [ ]:
base_path = "/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/three_staples_remvd"
output_paths_three= export_dna_structures(new_dna_structures_three, base_path)

In [ ]:
file_dir = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd/structure_1'
structure_id = os.path.basename(file_dir)
print(structure_id)


In [26]:
# Define the base simulation path
eq_steps = 1e5
prod_steps = 1e5
rel_steps = 1e3
    
# Parameters for simulations
eq_parameters = {'dt':f'0.003','steps':f'{eq_steps}','print_energy_every': f'1e5', 'interaction_type': 'DNA2',
                 'print_conf_interval':f'1e5', 'fix_diffusion':'false', 'T':f'20C','max_density_multiplier':f'50'}

prod_parameters = {'dt':f'0.003','steps':f'{prod_steps}','print_energy_every': f'1e5', 'interaction_type': 'DNA2',
                   'print_conf_interval':f'1e5', 'fix_diffusion':'false', 'T':f'20C','max_density_multiplier':f'50'}
rel_parameters = {'steps': f'{rel_steps}', 'max_backbone_force': '200', 'max_backbone_force_far': '200'}

sim_base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/metabackbone/notebooks/Simulations/simulations/Automatically_rmvd_staples/one_staple_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd'


# Function to queue relaxation simulations
def queue_relaxation_simulations(structures, base_path, sim_base_path):
    simulation_manager = SimulationManager()
    sim_list_rel = []
    for structure_id, structure in enumerate(structures):
        file_dir = os.path.join(base_path, f'structure_{structure_id}')
        sim_path = os.path.join(sim_base_path, f'1512_bp_{structure_id}')
        rel_dir = os.path.join(sim_path, 'relaxed')
        if not os.path.exists(file_dir):
            print(f"Directory does not exist: {file_dir}")
            continue
        sim_relax = Simulation(file_dir, rel_dir)
        sim_relax.build(clean_build='force')
        sim_relax.input.swap_default_input("cpu_MC_relax")
        sim_relax.input_file(rel_parameters)
        simulation_manager.queue_sim(sim_relax)
        sim_list_rel.append(sim_relax)
        print(f"Queued relaxation simulation for structure {structure_id}")
    simulation_manager.worker_manager(gpu_mem_block=False)
    print("Completed all relaxation simulations")
    return sim_list_rel

In [27]:
# Function to queue equilibration simulations
def queue_equilibration_simulations(structures, base_path, sim_base_path):
    simulation_manager = SimulationManager()
    sim_list_eq = []
    for structure_id, structure in enumerate(structures):
        sim_path = os.path.join(sim_base_path, f'1512_bp_{structure_id}')
        rel_dir = os.path.join(sim_path, 'relaxed')
        eq_dir = os.path.join(sim_path, 'eq')
        if not os.path.exists(rel_dir):
            print(f"Directory does not exist: {rel_dir}")
            continue
        sim_eq = Simulation(rel_dir, eq_dir)
        sim_eq.build(clean_build='force')
        sim_eq.input_file(eq_parameters)
        simulation_manager.queue_sim(sim_eq)
        sim_list_eq.append(sim_eq)
        print(f"Queued equilibration simulation for structure {structure_id}")
    simulation_manager.worker_manager(gpu_mem_block=False)
    print("Completed all equilibration simulations")
    return sim_list_eq

In [28]:
# Function to queue production simulations
def queue_production_simulations(structures, base_path, sim_base_path):
    simulation_manager = SimulationManager()
    sim_list_prod = []
    for structure_id, structure in enumerate(structures):
        sim_path = os.path.join(sim_base_path, f'1512_bp_{structure_id}')
        eq_dir = os.path.join(sim_path, 'eq')
        prod_dir = os.path.join(sim_path, 'prod')
        if not os.path.exists(eq_dir):
            print(f"Directory does not exist: {eq_dir}")
            continue
        sim_prod = Simulation(eq_dir, prod_dir)
        sim_prod.build(clean_build='force')
        sim_prod.input_file(prod_parameters)
        simulation_manager.queue_sim(sim_prod)
        sim_list_prod.append(sim_prod)
        print(f"Queued production simulation for structure {structure_id}")
    simulation_manager.worker_manager(gpu_mem_block=False)
    print("Completed all production simulations")
    return sim_list_prod

In [29]:
# Function to run all simulations
def run_all_simulations(structures, base_path, sim_base_path):
    print("Starting relaxation simulations...")
    sim_list_rel = queue_relaxation_simulations(structures, base_path, sim_base_path)
    print("Starting equilibration simulations...")
    sim_list_eq = queue_equilibration_simulations(structures, base_path, sim_base_path)
    print("Starting production simulations...")
    sim_list_prod = queue_production_simulations(structures, base_path, sim_base_path)
    return sim_list_rel, sim_list_eq, sim_list_prod

In [30]:
def main_one_strand(base_path, sim_base_path, left_indices, right_indices, rel_parameters, eq_parameters, prod_parameters, target_angle=90, tolerance=5):
    # Load the initial DNA structure
    print("Loading initial DNA structure...")
    initial_dna = load_dna_structure_files(base_path)
    
    results = []
    sphere_radius = 3.5  

    # Find the longest strand in the initial DNA structure
    print("Finding the longest strand in the DNA structure...")
    longest_strand, _ = find_longest_strand(initial_dna)

    # Find a valid point as the center of the sphere
    print("Finding a valid point P as the center of the sphere...")
    point = find_valid_point(initial_dna, left_indices, right_indices, longest_strand)
    print(f"Point P found at: {point}")

    # Find all bases (staples) within the sphere around the point
    print(f"Finding all bases within a sphere of radius {sphere_radius} around point P...")
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(initial_dna, point, sphere_radius)

    # Create a list of unique strands (staples) within the sphere
    strands_in_sphere = set(base_to_strand_mapping.values())
    print(f"Number of unique strands (staples) found in the sphere: {len(strands_in_sphere)}")

    for strand_index in strands_in_sphere:
        print(f"Processing strand index: {strand_index}")
        
        # Create a new structure by removing the current strand
        strand_list = [strand for idx, strand in enumerate(initial_dna.strands) if idx != strand_index]
        new_dna_structure = DNAStructure(strand_list, initial_dna.time, initial_dna.box, initial_dna.energy)

        # Export the new structure and run simulations
        print("Exporting the new structure...")
        output_paths = export_dna_structures([new_dna_structure], base_path)
        
        print("Running all simulations (relaxation, equilibration, production)...")
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_dna_structure], base_path, sim_base_path, rel_parameters, eq_parameters, prod_parameters)
        
        # Find the bend angle of the new structure
        print("Finding the bend angle of the new structure...")
        point_pos, bend_angle = find_bend_angle(new_dna_structure, left_indices, right_indices, longest_strand)
        print(f"Bend angle: {bend_angle} at point {point_pos}")
        
        results.append((point_pos, bend_angle))
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
    
    print("Finished processing all strands.")
    return results

In [31]:
results = main_one_strand(base_path, sim_base_path, left_indices, right_indices, rel_parameters, eq_parameters, prod_parameters)
print("Final Results:")
for result in results:
    print(result)

Loading initial DNA structure...


AssertionError: 

In [ ]:
def main_one_strand(base_path, sim_base_path, target_angle=90, tolerance=5):
    
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5  
    excluded_strands = set()

    longest_strand, _ = find_longest_strand(initial_dna)

    while True:
        point = find_valid_point(initial_dna, left_indices, right_indices, longest_strand)
        new_structure, removed_strand = remove_one_strand_in_sphere(initial_dna, point, sphere_radius, excluded_strands)
        # If no new structure is generated, exit the loop
        if new_structure is None:
            break
        
        # Add the removed strand to the set of excluded strands
        excluded_strands.add(removed_strand)
        
        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
    
    return results



In [ ]:
def main_two_strand(base_path, sim_base_path, target_angle=90, tolerance=5):
    
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5  
    excluded_strands = set()

    longest_strand, _ = find_longest_strand(initial_dna)

    while True:
        point = find_valid_point(initial_dna, left_indices, right_indices, longest_strand)
        new_structure, removed_strand = remove_two_strands_in_sphere(initial_dna, point, sphere_radius, excluded_strands)
        # If no new structure is generated, exit the loop
        if new_structure is None:
            break
        
        # Add the removed strand to the set of excluded strands
        excluded_strands.add(removed_strand)
        
        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
    
    return results

In [ ]:
def main_three_strand(base_path, sim_base_path, target_angle=90, tolerance=5):
    
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5  
    excluded_strands = set()

    longest_strand, _ = find_longest_strand(initial_dna)

    while True:
        point = find_valid_point(initial_dna, left_indices, right_indices, longest_strand)
        new_structure, removed_strand = remove_three_strands_in_sphere(initial_dna, point, sphere_radius, excluded_strands)
        # If no new structure is generated, exit the loop
        if new_structure is None:
            break
        
        # Add the removed strand to the set of excluded strands
        excluded_strands.add(removed_strand)
        
        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
    
    return results

In [ ]:
def main_iterative(base_path, sim_base_path, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5  
    excluded_strands = set()
    removed_strands_combination = []

    longest_strand, left_indices, right_indices = find_longest_strand(initial_dna)
    current_dna = initial_dna

    while True:
        point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        new_structure, removed_strand = remove_one_strand_in_sphere(current_dna, point, sphere_radius, excluded_strands)
        
        # If no new structure is generated, exit the loop
        if new_structure is None:
            break
        
        # Add the removed strand to the set of excluded strands and combination list
        excluded_strands.add(removed_strand)
        removed_strands_combination.append(removed_strand)
        
        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Print the details of the removed strand and current combination
        print(f"Removed strand: {removed_strand}")
        print(f"Current combination of removed strands: {removed_strands_combination}")
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
        
        # Update current_dna to the new structure for the next iteration
        current_dna = new_structure
    
    return results

In [ ]:
def main_iterative_two_staples(base_path, sim_base_path, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5  
    excluded_strands = set()
    removed_strands_combination = []

    longest_strand, left_indices, right_indices = find_longest_strand(initial_dna)
    current_dna = initial_dna

    while True:
        # Remove the first strand
        point1 = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        new_structure1, removed_strand1 = remove_one_strand_in_sphere(current_dna, point1, sphere_radius, excluded_strands)
        
        # If no new structure is generated, exit the loop
        if new_structure1 is None:
            break
        
        # Add the removed strand to the set of excluded strands and combination list
        excluded_strands.add(removed_strand1)
        removed_strands_combination.append(removed_strand1)

        # Remove the second strand
        point2 = find_valid_point(new_structure1, left_indices, right_indices, longest_strand)
        new_structure2, removed_strand2 = remove_one_strand_in_sphere(new_structure1, point2, sphere_radius, excluded_strands)
        
        # If no new structure is generated, exit the loop
        if new_structure2 is None:
            break

        # Add the second removed strand to the set of excluded strands and combination list
        excluded_strands.add(removed_strand2)
        removed_strands_combination.append(removed_strand2)

        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure2], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure2], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure2, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Print the details of the removed strands and current combination
        print(f"Removed strands: {removed_strand1}, {removed_strand2}")
        print(f"Current combination of removed strands: {removed_strands_combination}")
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
        
        # Update current_dna to the new structure for the next iteration
        current_dna = new_structure2
    
    # Handle case where only one staple is left to be removed
    if current_dna is not None:
        point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        final_structure, removed_strand = remove_one_strand_in_sphere(current_dna, point, sphere_radius, excluded_strands)
        
        if final_structure is not None:
            output_paths = export_dna_structures([final_structure], base_path)
            sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([final_structure], base_path, sim_base_path)
            
            point_pos, bend_angle = find_bend_angle(final_structure, left_indices, right_indices, longest_strand)
            results.append((point_pos, bend_angle))
            
            # Print the details of the removed strand and current combination
            removed_strands_combination.append(removed_strand)
            print(f"Removed strand: {removed_strand}")
            print(f"Current combination of removed strands: {removed_strands_combination}")
            
            if abs(bend_angle - target_angle) <= tolerance:
                print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                return results

    return results

In [ ]:
def main_remove_three_staples(base_path, sim_base_path, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5
    excluded_strands = set()
    removed_strands_combination = []

    longest_strand, left_indices, right_indices = find_longest_strand(initial_dna)
    current_dna = initial_dna

    while True:
        point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        new_structures, removed_strands_info = remove_three_strands_in_sphere(current_dna, point, sphere_radius)
        
        # If no new structure is generated, exit the loop
        if not new_structures:
            break
        
        # Loop through each new structure generated by removing three strands
        for new_structure, removed_strands in zip(new_structures, removed_strands_info):
            for removed_strand in removed_strands:
                excluded_strands.add(removed_strand)
                removed_strands_combination.append(removed_strand)
            
            # Export the new structure and run simulations
            output_paths = export_dna_structures([new_structure], base_path)
            sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
            
            # Find the bend angle of the new structure
            point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
            results.append((point_pos, bend_angle))
            
            # Print the details of the removed strands and current combination
            print(f"Removed strands: {removed_strands}")
            print(f"Current combination of removed strands: {removed_strands_combination}")
            
            # Check if the bend angle is within the desired tolerance
            if abs(bend_angle - target_angle) <= tolerance:
                print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                return results
            
            # Update current_dna to the new structure for the next iteration
            current_dna = new_structure

    return results


In [ ]:
# The provided code should work for any number of staples.
def main_remove_three_staples(base_path, sim_base_path, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5
    excluded_strands = set()
    removed_strands_combination = []

    longest_strand, left_indices, right_indices = find_longest_strand(initial_dna)
    current_dna = initial_dna

    while True:
        point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        new_structures, removed_strands_info = remove_three_strands_in_sphere(current_dna, point, sphere_radius)
        
        # If no new structure is generated, exit the loop
        if not new_structures:
            break
        
        # Loop through each new structure generated by removing three strands
        for new_structure, removed_strands in zip(new_structures, removed_strands_info):
            for removed_strand in removed_strands:
                excluded_strands.add(removed_strand)
                removed_strands_combination.append(removed_strand)
            
            # Export the new structure and run simulations
            output_paths = export_dna_structures([new_structure], base_path)
            sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
            
            # Find the bend angle of the new structure
            point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
            results.append((point_pos, bend_angle))
            
            # Print the details of the removed strands and current combination
            print(f"Removed strands: {removed_strands}")
            print(f"Current combination of removed strands: {removed_strands_combination}")
            
            # Check if the bend angle is within the desired tolerance
            if abs(bend_angle - target_angle) <= tolerance:
                print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                return results
            
            # Update current_dna to the new structure for the next iteration
            current_dna = new_structure

    # Handle cases where fewer than three staples are left
    remaining_strands = set(range(len(current_dna.strands))) - excluded_strands
    while remaining_strands:
        if len(remaining_strands) >= 3:
            point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
            new_structures, removed_strands_info = remove_three_strands_in_sphere(current_dna, point, sphere_radius)
            for new_structure, removed_strands in zip(new_structures, removed_strands_info):
                for removed_strand in removed_strands:
                    excluded_strands.add(removed_strand)
                    removed_strands_combination.append(removed_strand)
                output_paths = export_dna_structures([new_structure], base_path)
                sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
                point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
                results.append((point_pos, bend_angle))
                print(f"Removed strands: {removed_strands}")
                print(f"Current combination of removed strands: {removed_strands_combination}")
                if abs(bend_angle - target_angle) <= tolerance:
                    print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                    return results
                current_dna = new_structure
        elif len(remaining_strands) == 2:
            point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
            new_structure, removed_strand1 = remove_one_strand_in_sphere(current_dna, point, sphere_radius, excluded_strands)
            point = find_valid_point(new_structure, left_indices, right_indices, longest_strand)
            final_structure, removed_strand2 = remove_one_strand_in_sphere(new_structure, point, sphere_radius, excluded_strands)
            if final_structure is not None:
                output_paths = export_dna_structures([final_structure], base_path)
                sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([final_structure], base_path, sim_base_path)
                point_pos, bend_angle = find_bend_angle(final_structure, left_indices, right_indices, longest_strand)
                results.append((point_pos, bend_angle))
                removed_strands_combination.extend([removed_strand1, removed_strand2])
                print(f"Removed strands: {removed_strand1}, {removed_strand2}")
                print(f"Current combination of removed strands: {removed_strands_combination}")
                if abs(bend_angle - target_angle) <= tolerance:
                    print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                    return results
                remaining_strands -= {removed_strand1, removed_strand2}
        elif len(remaining_strands) == 1:
            point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
            final_structure, removed_strand = remove_one_strand_in_sphere(current_dna, point, sphere_radius, excluded_strands)
            if final_structure is not None:
                output_paths = export_dna_structures([final_structure], base_path)
                sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([final_structure], base_path, sim_base_path)
                point_pos, bend_angle = find_bend_angle(final_structure, left_indices, right_indices, longest_strand)
                results.append((point_pos, bend_angle))
                removed_strands_combination.append(removed_strand)
                print(f"Removed strand: {removed_strand}")
                print(f"Current combination of removed strands: {removed_strands_combination}")
                if abs(bend_angle - target_angle) <= tolerance:
                    print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                    return results
                remaining_strands.remove(removed_strand)
    return results

# Example usage
# base_path and sim_base_path should be set to the appropriate directories
results = main_remove_three_staples(base_path, sim_base_path)
visualize_results(results)


In [ ]:
# code for iteratively removing two staples at a time from the DNA structure, handling any number of staples
def main_remove_two_staples(base_path, sim_base_path, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5
    excluded_strands = set()
    removed_strands_combination = []

    longest_strand, left_indices, right_indices = find_longest_strand(initial_dna)
    current_dna = initial_dna

    while True:
        point1 = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        new_structure1, removed_strand1 = remove_one_strand_in_sphere(current_dna, point1, sphere_radius, excluded_strands)
        
        # If no new structure is generated, exit the loop
        if new_structure1 is None:
            break
        
        # Add the removed strand to the set of excluded strands
        excluded_strands.add(removed_strand1)
        removed_strands_combination.append(removed_strand1)

        point2 = find_valid_point(new_structure1, left_indices, right_indices, longest_strand)
        new_structure2, removed_strand2 = remove_one_strand_in_sphere(new_structure1, point2, sphere_radius, excluded_strands)
        
        # If no new structure is generated, exit the loop
        if new_structure2 is None:
            break

        # Add the second removed strand to the set of excluded strands
        excluded_strands.add(removed_strand2)
        removed_strands_combination.append(removed_strand2)

        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure2], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure2], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure2, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Print the details of the removed strands and current combination
        print(f"Removed strands: {removed_strand1}, {removed_strand2}")
        print(f"Current combination of removed strands: {removed_strands_combination}")
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
        
        # Update current_dna to the new structure for the next iteration
        current_dna = new_structure2

    # Handle case where only one staple is left to be removed
    remaining_strands = set(range(len(current_dna.strands))) - excluded_strands
    if len(remaining_strands) == 1:
        point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
        final_structure, removed_strand = remove_one_strand_in_sphere(current_dna, point, sphere_radius, excluded_strands)
        
        if final_structure is not None:
            output_paths = export_dna_structures([final_structure], base_path)
            sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([final_structure], base_path, sim_base_path)
            
            point_pos, bend_angle = find_bend_angle(final_structure, left_indices, right_indices, longest_strand)
            results.append((point_pos, bend_angle))
            
            removed_strands_combination.append(removed_strand)
            print(f"Removed strand: {removed_strand}")
            print(f"Current combination of removed strands: {removed_strands_combination}")
            
            if abs(bend_angle - target_angle) <= tolerance:
                print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                return results

    return results

# Example usage
# base_path and sim_base_path should be set to the appropriate directories
results = main_remove_two_staples(base_path, sim_base_path)
visualize_results(results)


In [ ]:
# def main_iterative(base_path, sim_base_path, target_angle=90, tolerance=5):
#     initial_dna = load_dna_structure_files(base_path)
#     results = []
#     sphere_radius = 3.5  
#     excluded_strands = set()

#     longest_strand, left_indices, right_indices = find_longest_strand(initial_dna)
#     current_dna = initial_dna

#     while True:
#         point = find_valid_point(current_dna, left_indices, right_indices, longest_strand)
#         new_structure, removed_strand = remove_one_strand_in_sphere(current_dna, point, sphere_radius, excluded_strands)
        
#         # If no new structure is generated, exit the loop
#         if new_structure is None:
#             break
        
#         # Add the removed strand to the set of excluded strands
#         excluded_strands.add(removed_strand)
        
#         # Export the new structure and run simulations
#         output_paths = export_dna_structures([new_structure], base_path)
#         sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
        
#         # Find the bend angle of the new structure
#         point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
#         results.append((point_pos, bend_angle))
        
#         # Check if the bend angle is within the desired tolerance
#         if abs(bend_angle - target_angle) <= tolerance:
#             print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
#             return results
        
#         # Update current_dna to the new structure for the next iteration
#         current_dna = new_structure
    
#     return results

In [ ]:
def visualize_results(results, target_angle=90):
    points, bend_angles = zip(*results)
    
    # Scatter plot of bend angles
    plt.figure(figsize=(10, 6))
    plt.scatter(range(len(bend_angles)), bend_angles, c='blue', label='Bend Angles')
    plt.axhline(y=target_angle, color='r', linestyle='--', label='Target Angle')
    plt.xlabel('Removal Step')
    plt.ylabel('Bend Angle')
    plt.title('Bend Angles After Each Staple Removal')
    plt.legend()
    plt.show()

    # Line plot of bend angles
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(bend_angles)), bend_angles, marker='o', label='Bend Angles')
    plt.axhline(y=target_angle, color='r', linestyle='--', label='Target Angle')
    plt.xlabel('Removal Step')
    plt.ylabel('Bend Angle')
    plt.title('Bend Angles After Each Staple Removal')
    plt.legend()
    plt.show()

In [ ]:
results = main(base_path, sim_base_path)
visualize_results(results)

In [ ]:
results = main_iterative(base_path, sim_base_path)
visualize_results(results)

In [ ]:
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/simulations_structure/Automatically_rmvd_staples/one_staple_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd'

run_all_simulations(new_dna_structures, base_path, sim_base_path)

In [ ]:
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/simulations_structure/Automatically_rmvd_staples/two_staples_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/two_staples_remvd'

run_all_simulations(new_dna_structures_two, base_path, sim_base_path)

In [ ]:
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/simulations_structure/Automatically_rmvd_staples/three_staples_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/three_staples_remvd'

run_all_simulations(new_dna_structures_three, base_path, sim_base_path)

In [ ]:
structures = [f'structure_{i}' for i in range(10)]  # Example list of structures
sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations(structures, base_path, sim_base_path)

In [ ]:
# Plot energy for all relaxation simulations
for sim in sim_list_rel:
    sim.analysis.plot_energy()

In [ ]:
# Plot energy for all equilibration simulations
for sim in sim_list_eq:
    sim.analysis.plot_energy()

In [ ]:
# Plot energy for all prod simulations
for sim in sim_list_prod:
    sim.analysis.plot_energy()